***AUTHOR : Manel Djehiche SII M2 promo 2019/2020 usthb***

***gmail  : maneldje81@gmail.com***

`Ce fichier contient tous les étapes de prétraitement avec la validation en utilisant le modèle Naive Bayes`

In [ ]:
pip install unidecode

     |████████████████████████████████| 245kB 4.5MB/s 


In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None 
import nltk
from nltk import re
from nltk import defaultdict
import io
from nltk import  word_tokenize
from nltk import  sent_tokenize
from nltk.corpus import stopwords
import string
from nltk import  word_tokenize
from nltk import  sent_tokenize
import codecs
import csv
import pickle
from unidecode import unidecode
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_numeric
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from itertools import groupby
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#vérifie si l'encodage est utf_8
def check_utf8(filename): 
        try:
            f = codecs.open(filename, encoding='utf-8', errors='strict')
            for line in f:
                pass
            return 1 #valid utf_8
        except UnicodeDecodeError:
            return 0 #invalid utf_8

# ouvrir un fichier text
def open_txt_file(chemin):
    #open text file atfter checking if its utf_8  encoding    
    check=check_utf8(chemin)
    if(check == 1):
        f = io.open(chemin,'r',encoding='utf8')
    elif(check == 0):
        f = io.open(chemin,'r')
    return f.read()

# convertir un text vers une liste
def text_to_list(chemin):
    text=open_txt_file(chemin)
    liste=[]
    liste=text.split("\n")
    return liste

# ouvrir un fichier pickle
def open_pickle(path):
  with open(path, 'rb') as handle: 
    doc= pickle.load(handle)
    return doc

def save_pickle(variable,filename):
    with open(filename, 'wb') as handle:
         pickle.dump(variable, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
      
        
def open_csv(path):
      dataset=pd.read_csv(path,error_bad_lines=False)
      dataset.columns = ['id_comment','comment','id_post','polarity']
      return dataset

def is_arabizi(word):
  if len(re.findall(r'^[a-zA-Z0-9]+$',word)) > 0:
    return 1
  else: 
    return 0

def tokenization(text):
    return word_tokenize(text)

def naive_bayes(data):
  X = data['text'].values
  Y = data['label'].values
  #-- split
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1000)

  #-- tfidf for text only not class
  vectorizer = TfidfVectorizer()
  vectorizer.fit_transform(X)
  x_train_tfidf = vectorizer.transform(x_train)
  x_test_tfidf = vectorizer.transform(x_test)
  
  #model
  gnb = ComplementNB()
  gnb.fit(x_train_tfidf,y_train)
  y_pred = gnb.predict(x_test_tfidf)

  # Model Accuracy, how often is the classifier correct?
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

def logistic_regression(data):
  X = data['text'].values
  Y = data['label'].values
  #-- split
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1000)

  #-- tfidf for text only not class
  vectorizer = TfidfVectorizer()
  vectorizer.fit_transform(X)
  x_train_tfidf = vectorizer.transform(x_train)
  x_test_tfidf = vectorizer.transform(x_test)
  
  #model
  model = LogisticRegression(solver = 'lbfgs')
  model.fit(x_train_tfidf,y_train)

  # use the model to make predictions with the test data
  y_pred = model.predict(x_test_tfidf)
  accuracy = metrics.accuracy_score(y_test, y_pred)
  print('Accuracy: {:.2f}'.format(accuracy))

## Functions de pré-traitement

In [ ]:
#Suppression URL, TAGS, HASHTAG
def romve_urls(text):
  return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
  
  
def remove_tags(text):
  return re.sub(r"@(\w+|_|\.)*", ' ', text, flags=re.MULTILINE)
  

def remove_hashtag(text):
  return re.sub(r"#(\w+|_)*", ' ', text, flags=re.MULTILINE)

#Convertion en miniscule + Suppression les lettres répérés
def remove_repeating_char(text):
  lower_word=text.lower()
  liste=tokenization(lower_word)
  sentence=''
  for word in liste:
      if(sentence == ''):
        new=''.join(c for c, _ in groupby(word))
        sentence=sentence+''+new
      else:
        new=''.join(c for c, _ in groupby(word))
        sentence=sentence+' '+new
  return sentence

#Suppression des emojis
url="/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/all_emojis.txt"
liste_emoji= text_to_list(url)
def remove_imogi(text):  
   x = str(text)
   for punct in liste_emoji:
      if punct in x:
          x = x.replace(punct, f'')
   return x

#Suppression de la ponctuations
def remove_ponctuation(text):
   puncts = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗',' ️�','▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
   
   x = str(text)
   for punct in puncts:
      if punct in x:
          x = x.replace(punct, f'')
   return x


#Suppression des chiffres ésolés
def remove_numbers(text):
  return  re.sub(r"\b[0-9]+\b", "",text)
  

#Suppression des repétitions et garder 1 occurence de chaque lettres max, sauf si le mot est anglais on le touche pas
def remove_short(text):
  lower_word=text.lower()
  liste=tokenization(lower_word)
  sentence=''
  for word in liste:
    if(len(word) == 1):
        new=''
        sentence=sentence+''+new
    else:
      if(sentence == ''):
        new=word
        sentence=sentence+''+new
      else:
        new=word
        sentence=sentence+' '+new
      
  return sentence

#Suppression de la ponctuations arabe
def remove_arabic_ponctuation(text):
   puncts = ['﴿', '﴾', '۩‎', '۞', '۝', '(', '-', '۔‎', '٭', "؟‎", '؞', '؛‎', '؏', '؎', '؍', '،', '؆', '؇', '؈', '؄', '؁', '؀']
   x = str(text)
   for punct in puncts:
      if punct in x:
          x = x.replace(punct, f'')
   return x
  
#normalisation texte arabe
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ي", "ى", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

#Suppression du Diactritics y compris el chadda et tatwil (l'alongation)
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text

#abreviation 
def abreviation(text):
  return re.findall(r'\b[^AEIOU_0-9-\u0621-\u064a\ufb50-\ufdff\ufe70-\ufefc\W]+\b', text, flags=re.I)

def abreviationne(text):
  liste=[]
  res=re.findall(r'\b[^AEIOU_0-9-\u0621-\u064a\ufb50-\ufdff\ufe70-\ufefc\W]+\b', text, flags=re.I)
  liste.extend(res)
  return liste

url="/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/abréviation.csv"
abreviation=pd.read_csv(url)

def replace_abrv(texte):
 word= tokenization(texte) 
 x=str(texte)
 ma_lis=[]
 for val in word:
   if(len(abreviationne(val))!=0):
     for indice  in list(abreviation.index.values): 
       text=str(abreviation['text'][indice])
       if(val== text):
         tt=str(abreviation['abreviation'][indice])
         ma_lis.append(tt)
         x = x.replace(val, tt)  
 return x  

#stop words
url="/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/stopwrd.txt"
stopwords= text_to_list(url)

def remove_stop_words(texte):
  word= tokenization(texte) 
  x=str(texte)
  ma_lis=[]
  for val in word:
    for text  in  stopwords: 
      if(val == text):
        x = x.replace(val,'')  
  return x  

all_dz_msa_stopwords=open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/all_dz_msa_stopwords.pickle")
def remove_stop_words_big(texte):

  word= tokenization(texte) 
  x=str(texte)
  ma_lis=[]
  for val in word:
    for text  in  all_dz_msa_stopwords: 
      if(val == text):
        x = x.replace(val,'')  
  return x  


In [ ]:
len(liste_emoji)

3364

# DT_1 

`Avant-prétraitement`

In [ ]:
donne=pd.read_csv('/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné trouvé/MatouiKadirCorpus.csv')

In [ ]:
#Suppression des instances doublons
donne=donne.drop_duplicates()
print(len(donne))

#supression nan
donne.isnull().sum()
donne=donne.dropna(how='any')
donne.isnull().sum()

16945


text     0
label    0
dtype: int64

# Pré-taitement initial

In [ ]:
#--------- nothing

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=romve_urls(text)
naive_bayes(donne)

Accuracy: 0.6600166469893743


In [ ]:
#--------- urls
for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=romve_urls(text)
naive_bayes(donne)

Accuracy: 0.6627556469893743


In [ ]:
#--------- tags

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_tags(text)
naive_bayes(donne)

Accuracy: 0.6629000149893332


In [ ]:
#--------- hashtags

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_hashtag(text)
naive_bayes(donne)

Accuracy: 0.6641265149735632


In [ ]:
#--------- repeating characteres

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_repeating_char(text)
naive_bayes(donne)

Accuracy: 0.6663958102000692


In [ ]:
#--------- ponctuation générale

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_ponctuation(text) 
naive_bayes(donne)

Accuracy: 0.6770029504132231


In [ ]:
#--------- emojis

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_imogi(text)
naive_bayes(donne)

Accuracy: 0.6770329505583631


In [ ]:
#--------- remove short word with 1 only caractère

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_short(text)
naive_bayes(donne)

Accuracy: 0.6770326585000689


In [ ]:
#-------- generale numbers

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_numbers(text)
naive_bayes(donne)


Accuracy: 0.6747343565525383


# Traitement des Abréviations

In [ ]:
#-------- abreviation

for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=replace_abrv(text)
naive_bayes(donne)


Accuracy: 0.6765053128689492


# Traitement multilinguisme

`Traduction`

In [ ]:
donnee_traduit= open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné trouvé/donnee_traduit.pickle")
donne_tra= pd.DataFrame.from_dict(donnee_traduit,orient='index')
donne=donne_tra

#-------- traduction
naive_bayes(donne_tra)


Accuracy: 0.68054325361677


`Phonetics`

In [ ]:
donnee_phonetics= open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné trouvé/donnee_phonetics.pickle")
donne_pho= pd.DataFrame.from_dict(donnee_phonetics,orient='index')
donne=donne_pho

#-------- generale phonetics
naive_bayes(donne_pho)


Accuracy: 0.6808385001476233


`Transliteration`

In [ ]:
donnee_yamli= open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné trouvé/donnee_yamli.pickle")

def is_arabizi2(text):
    l = len(re.findall(r'[A-Za-z0-9]+',text))
    if(l > 0):
      return ''
    else:
      return text

def remove_fr(texte):
   word= tokenization(texte) 
   for val in word:
     texte = texte.replace(val, is_arabizi2(val))
   return texte
  

for x in donnee_yamli.keys():
  donnee_yamli[x]['text']=' '.join(donnee_yamli[x]['text'])
  donnee_yamli[x]['text']=remove_fr(donnee_yamli[x]['text'])

donne_yam= pd.DataFrame.from_dict(donnee_yamli,orient='index')
donne=donne_yam

#-------- generale transliteration
naive_bayes(donne)

Accuracy: 0.68683850013216846


In [ ]:
donne_yam['text'][16524]

'لالكراك ربراب ىجب ان ىكون جىدا ست راس الافعى حلمه الثدى كل هذه الاموال خلال سنىه فى اسود كنفىنس مع هذا رل كل واحد كل واحد سبسون'

# Traitement de l'arabe

`Arabic preprocessing`

In [ ]:
#---- remove arabic ponctuation
for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_arabic_ponctuation(text)
naive_bayes(donne)

Accuracy: 0.68698521013200006


In [ ]:
#---- normalisation arabic
for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=normalize_arabic(text)
naive_bayes(donne)

Accuracy: 0.68908001039023522


In [ ]:
#---- diacritics arabic
for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_diacritics(text)
naive_bayes(donne)

Accuracy: 0.69891437850605254


In [ ]:
save_pickle(donne,"/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné téléchargé/data_ar_final_2.pickle")

# Traitement des mots vides

`Liste 1`

In [ ]:
#---- stop word 
for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_stop_words(text)
naive_bayes(donne)

Accuracy: 0.6699143785060525


In [ ]:
save_pickle(donne,"/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné téléchargé/data_ar_final_1.pickle")

**Liste 2**

In [ ]:
#---- stop word 
for indice in donne.index: 
    text=str(donne['text'][indice])
    donne['text'][indice]=remove_stop_words_big(text)
naive_bayes(donne)


Accuracy: 0.6666666666666666
